In [22]:
import os
from llama_index.llms.perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv()

pplx_api_key = os.getenv("PERPLEXITY_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

llm = Perplexity(
    api_key=pplx_api_key, model="mistral-7b-instruct", temperature=0.5
)

In [20]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [24]:
from llama_index.core.indices.knowledge_graph.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [28]:
from llama_index.core.llms import ChatMessage

query = "what are the deathly hallows"

response = query_engine_with_nl2graphquery.query(
    query,
)

if response.metadata is None:
    # 1) answer needs to be queried from Llama2 endpoint
    messages_dict = [
        {"role": "system", "content": "Be precise and concise."},
        {"role": "user", "content": query},
    ]
    messages = [ChatMessage(**msg) for msg in messages_dict]

    external_response = llm.chat(messages)
    print(external_response)

    # 2) answer needs to be stored into txt file for loading into KG
    with open('data/external_response.txt', 'w') as f:
        print(external_response, file=f)
else:
    print("From Graph: " + str(response))
    print(list(list(response.metadata.values())[0]["kg_rel_map"].keys()))

assistant: The Deathly Hallows are a trio of magical objects in J.K. Rowling's Harry Potter series: a hollowed-out Elder Wand, a Resurrection Stone, and a Cloak of Invisibility. These artifacts are known as the "Hallows" because they bring the wielder closer to immortality. They are important plot elements in the seventh book, "Harry Potter and the Deathly Hallows," as Harry, Ron, and Hermione seek to destroy Voldemort and end the war between the Death Eaters and the Order of the Phoenix.


In [30]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['data/external_response.txt']
).load_data()

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

In [35]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query(query)
print(response)

The Deathly Hallows are three magical objects in the Harry Potter series: the Elder Wand, the Resurrection Stone, and the Cloak of Invisibility.
